<a href="https://colab.research.google.com/github/comfortjah/FLiT-Data-Science-Mentorship-Projects/blob/main/MarketBasketAnalysis_Flitproject1_2023_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market Basket Analysis

## FLiT Apprenticeship First Project

### Importing Python Libraries.

In [ ]:
import numpy as np # Lib for linear algebra
import pandas as pd # lib for data processing
import seaborn as sns # lib for plotting graphs
import matplotlib.pyplot as plt # lib also for plotting
import io
from google.colab import drive
drive.mount('/content/drive')

### Importing the Dataset and reading the first five rows.

In [ ]:
!pwd
df =pd.read_csv("drive/My Drive/Colab Notebooks/Market Basket Analysis - Groceries_dataset.csv")
df.head()

### Getting the Data overview.

In [ ]:
df.info()

### Checking for duplicate Values

In [ ]:
if (duplicated := df.duplicated(keep=False)).any():
    some_duplicates = df[duplicated].sort_values(by=df.columns.to_list()).head()
    print(f"Dataframe has one or more duplicated rows, for example:\n{some_duplicates}")

### Checking for Null values.

In [ ]:
df.isnull().sum().sort_values(ascending = False)

### Converting the date datatype

In [ ]:
df['date'] = pd.to_datetime(df["Date"])
df.info()

In [ ]:
df.head()

### Plotting a Bar Chart for Item Distribution

In [ ]:
Item_distr = df.groupby( by = 'itemDescription').size().reset_index(name = 'Frequency').sort_values(by = 'Frequency', ascending = False).head(10)

# Declaring Variables
bars = Item_distr["itemDescription"]
height = Item_distr["Frequency"]
x_pos = np.arange(len(bars))

# Bar chart Discription

plt.figure(figsize = (16,9))

plt.bar(x_pos, height, color=(0.3,0.4,0.5,0.4))

plt.title("Top 10 Sold Items")

plt.xlabel("Name of Item")
plt.ylabel("Quantity Sold")

plt.xticks(x_pos, bars)

plt.show()





### Setting Date as the Index

In [ ]:
df_date = df.set_index(['Date'])
df_date

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['date']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'date'}, axis=1)
              .sort_values('date', ascending=True))
  xs = counted['date']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df_date.sort_values('date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('count()')

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  from matplotlib import pyplot as plt
  import seaborn as sns
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['date']
  ys = series['Member_number']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df_date.sort_values('date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('Member_number')

In [ ]:
from matplotlib import pyplot as plt
df_date['Member_number'].plot(kind='hist', bins=20, title='Member_number')
plt.gca().spines[['top', 'right',]].set_visible(False)

### Plotting a Line graph

In [ ]:
cust_level = df[["Member_number", "itemDescription"]].sort_values(by = "Member_number", ascending = False)
cust_level["itemDescription"] = cust_level["itemDescription"].str.strip()
cust_level

In [ ]:
transactions = [a[1]["itemDescription"].tolist() for a in list(cust_level.groupby(["Member_number"]))]

In [ ]:
!pip install apyori
from apyori import apriori
rules = apriori(transactions = transactions, min_support = 0.002, min_confidence = 0.05, min_lift = 3, min_lenght = 2)

In [ ]:
results = list(rules)

In [ ]:
results

In [ ]:
def inspect(results):
    lhs = [tuple(result[2][0][0])[0] for result in results]
    rhs = [tuple(result[2][0][1])[0] for result in results]
    supports = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

Ridf = pd.DataFrame(inspect(results), columns = ['Left hand side', 'Right hand side', 'Support', 'Confidences', 'Lifts'])


In [ ]:
Ridf.nlargest(n = 10, columns = "Lift")